In [1]:
import gym
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from collections import deque

In [2]:
# keras model approach
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dense, Input
from tensorflow.keras.activations import relu

In [3]:
# ADDING THE CODE SO THAT TENSORFLOW DOES NOT EAT THE WHOLE GPU MEMORY
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.set_floatx('float32')

In [4]:
env = gym.make('CartPole-v1')
env.action_space

Discrete(2)

In [5]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [6]:
def model_keras():
    
    inputs = Input(shape=(4,))
    
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(inputs)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    output = Dense(2,activation='linear',kernel_initializer="glorot_uniform")(x)
    model = Model(inputs=inputs, outputs=output, name="RL_Value_Function")
    
    print(model.summary())
    
    model.compile(optimizer=adam,loss='mean_squared_error',metrics=['mean_squared_error'])
    
    return model
def model_keras():    
    model = Sequential()
    model.add(Dense(24, input_dim=4, activation='relu',
                    kernel_initializer='he_uniform'))
    model.add(Dense(24, activation='relu',
                    kernel_initializer='he_uniform'))
    model.add(Dense(2, activation='linear',
                    kernel_initializer='he_uniform'))
    print(model.summary())
    model.compile(loss='mse', optimizer='adam')
    return model
# model 2 is the target model
model_1 = model_keras()
model_2 = model_keras()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_______________________________

In [7]:
def custom_loss(y_true,y_pred):
    return tf.keras.losses.mean_squared_error(y_true,y_pred)

In [8]:
# lets build some memory into the model to perform decorrelated batch updates
# this is TD learning

# so apparantly the NN has to be adjusted only according to the action taken by it
# for example if action 2 is taken then only the weights for action 2 should be changed
# for this reason we should only update the q_2 vector with regarding the chosen action
# rewards will also be added to that action only
# q_2 is what we thought the value of the state will be after doing action
# we also add the reward and make this the target for the NN

replay_batch = deque(maxlen = 3000)

warmup = 10 #will start training after these many episodes have passed

# to balance exploration
epsilon = {
"epsilon" : 1.0,
"epsilon_decay": 0.999,
"epsilon_min":0.01,
}

In [9]:
import random
random.seed(2020)

#@tf.function
def batch_train(model_1,model_2,gamma,batch_size,epsilon):
    
    #decaying the exploration
    if epsilon['epsilon'] > epsilon['epsilon_min']:
         epsilon['epsilon'] =  epsilon['epsilon'] * epsilon['epsilon_decay']
   
    batch = random.sample(replay_batch,batch_size)
    
    batch_reward = []
    batch_action = []
    batch_done = []
    
    batch_current_state = np.zeros((batch_size, 4))
    batch_next_state = np.zeros((batch_size, 4))

    for i in range(batch_size):
        batch_reward.append(batch[i][2])
        batch_action.append(batch[i][1])
        batch_current_state[i] = batch[i][0]
        batch_next_state[i] = batch[i][3]
        batch_done.append(batch[i][4])
    #lets calculate the next state value as the current value will be calculated in 
    # in gradient tape
    
    next_q = model_2.predict(batch_next_state)
    
    max_q = []
    for i in next_q:
        max_q.append(max(i))
    max_q = np.array(max_q,dtype = 'float32')
    
    target = batch_reward + gamma*max_q # this is the Q learning Target
    
    with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model_1(batch_current_state, training=True)
        
        q_target = np.array(logits)
        
        # VERY IMPORTANT NOTE, IF THE EPISODE ENDS THE DONE VALUE BECOMES TRUE
        # IT IS VERY IMPORTANT THAT THE NN UPDATES TOWARDS THIS TRUE VALUE RATHER THAN
        # ITS OWN THINKING VALUE (r + gamma*max(action)) THAT WE USE FOR ALL
        # NON TERMINAL REWARDS 
        # THIS MAKES OR BREAKS THE NETWORK VERY VERY IMPORTANT
        for i in range(batch_size):
            q_target[i][batch_action[i]] = target[i]
            if batch_done[i]:
                q_target[i][batch_action[i]] = batch_reward[i]
        
        # calculating the loss
        loss_value = custom_loss(q_target,logits)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model_1.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    adam.apply_gradients(zip(grads, model_1.trainable_weights))

def policy(q_vals,eps):
    # lets implement a policy which decays
    if np.random.rand() <= eps:  
        return random.randrange(2)
    else:
        action = np.argmax(q_vals[0])
        return action
def update_target_network():
    model_2.set_weights(model_1.get_weights())

In [10]:
global_steps = 0
# to have same networks in the starting
update_target_network()

for i in tqdm(range(150)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    done = False
    total_reward = 0
    while not done:        
        #storing the current state
        state_1 = observation
        
        # this is the current q values
        q_state = model_1(observation)
        
        action = policy(q_state,epsilon["epsilon"]) # current action
        observation,reward,done,info = env.step(action)
        
        # calculating the total reward
        total_reward = total_reward + reward

        observation = np.expand_dims(observation, axis=0)
        state_2 = observation
        state_reward = reward
        
        replay_batch.append((state_1,action,state_reward,state_2,done))
        
        if i>warmup:
            batch_train(model_1,model_2,0.99,64,epsilon)
            global_steps = global_steps + 1
            
        if done:
            update_target_network()
            
    print(total_reward)

  0%|          | 0/150 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



  4%|▍         | 6/150 [00:00<00:23,  6.00it/s]

21.0
21.0
35.0
15.0
24.0
17.0
12.0
13.0
25.0
24.0
15.0


  8%|▊         | 12/150 [00:01<00:20,  6.65it/s]

...

500.0


 99%|█████████▉| 149/150 [07:19<00:09,  9.12s/it]

80.0


100%|██████████| 150/150 [07:22<00:00,  2.95s/it]

114.0


In [ ]:
env.close()

In [11]:
#lets test the nn
for i in tqdm(range(500)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    total_reward = 0
    for j in range(1000):
        env.render()
        nn_out = model_2.predict(observation)
        action = policy(nn_out,0)
        print(nn_out[0])
        print(action)
        observation,reward,done,info = env.step(action)
        observation = np.expand_dims(observation, axis=0)
        total_reward = total_reward + reward
        if done:
            print("episode ended")
            break
env.close()


  0%|          | 0/500 [00:00<?, ?it/s]

[79.41802  80.228676]
1
[79.88904 77.98007]
0
[79.7922  80.12785]
1
[79.79381 77.46305]
0
[80.15454 79.97158]
0
[76.915276 79.88707 ]
1
[80.14157 79.75447]
0
[77.2135  79.88257]
1
[80.132355 79.553185]
0
[77.49714  79.878746]
1
[80.12646 79.3618 ]
0
[77.772896 79.87449 ]
1
[80.119484 79.18199 ]
0
[78.04724 79.86868]
1
[80.11024  79.010155]
0
[78.3265  79.86023]
1
[80.10439 78.83025]
0
[78.61708 79.84793]
1
[80.101654 78.6372  ]
0
[78.925575 79.830536]
1
[80.101776 78.425606]
0
[79.25896 79.80666]
1
[80.05612  78.105095]
0
[79.61746 79.76999]
1
[79.99204 77.71726]
0
[79.63201 79.47102]
0
[76.699646 79.40876 ]
1
[79.62061 79.35282]
0
[76.95575 79.51327]
1
[79.612595 79.26398 ]
0
[77.18036 79.6185 ]
1
[79.60743 79.20015]
0
[77.3781  79.72436]
1
[79.60468  79.157646]
0
[77.55287 79.83075]
1
[79.60397 79.13336]
0
[77.70793  79.937584]
1
[79.60499 79.12472]
0
[77.846054 80.04482 ]
1
[79.60745 79.12955]
0
[77.96952 80.15243]
1
[79.611115 79.14604 ]
0
[78.080246 80.260315]
1
[79.61579 79.17271


  0%|          | 1/500 [00:09<1:18:40,  9.46s/it]

[59.57892 57.07643]
0
[57.181557 56.835228]
0
[53.943108 55.065994]
1
[56.70844 56.05567]
0
episode ended
[79.69784 80.28251]
1
[79.76599  77.879166]
0
[79.91855 80.19908]
1
[79.708336 77.511795]
0
[80.1535  80.07321]
0
[76.818245 79.979546]
1
[80.14122 79.976  ]
0
[76.97207 79.99902]
1
[80.125854 79.90497 ]
0
[77.10804 80.01513]
1
[80.11358  79.845505]
0
[77.229546 80.03298 ]
1
[80.10392 79.79543]
0
[77.33937 80.05222]
1
[80.0965  79.75278]
0
[77.439926 80.07247 ]
1
[80.09099  79.715836]
0
[77.53343 80.09341]
1
[80.08713  79.683044]
0
[77.62183 80.11472]
1
[80.08468  79.652954]
0
[77.70699  80.136116]
1
[80.08346 79.62423]
0
[77.7906   80.157326]
1
[80.08333  79.595566]
0
[77.87436 80.17804]
1
[80.08415  79.565704]
0
[77.95988  80.197975]
1
[80.08583 79.53332]
0
[78.04885 80.21683]
1
[80.08828 79.4971 ]
0
[78.143005 80.234245]
1
[80.09146  79.455605]
0
[78.2442  80.24986]
1
[80.09532 79.40732]
0
[78.35444  80.263275]
1
[80.09986 79.35054]
0
[78.47591 80.27404]
1
[80.10505  79.283424]



  0%|          | 2/500 [00:17<1:15:56,  9.15s/it]


1
episode ended
[79.15304 80.16695]
1
[79.77528  78.412544]
0
[79.16611 80.13332]
1
[79.77285 78.30936]
0
[79.22415 80.09148]
1
[79.75895 78.15487]
0
[79.328316 80.040886]
1
[79.73356  77.946335]
0
[79.48089  79.980835]
1
[79.69636  77.679665]
0
[79.68532  79.910324]
1
[79.64673 77.34941]
0
[79.94636 79.82817]
0
[76.550316 79.629135]
1
[80.03464 79.82083]
0
[76.67871 79.67273]
1
[80.0266  79.76962]
0
[76.78438 79.71692]
1
[80.01999 79.73715]
0
[76.869644 79.76161 ]
1
[80.01026  79.729225]
0
[76.93633 79.80679]
1
[80.0026  79.73477]
0
[76.985756 79.85247 ]
1
[79.996635 79.75303 ]
0
[77.0188 79.8986]
1
[79.99205  79.783615]
0
[77.03592 79.9452 ]
1
[79.98857 79.82644]
0
[77.03716 79.99232]
1
[79.986   79.88174]
0
[77.02218 80.03993]
1
[79.98408 79.95002]
0
[76.99025 80.08812]
1
[79.98264  80.032166]
1
[79.65787 77.46147]
0
[80.01593 79.8961 ]
0
[77.10198 80.10908]
1
[80.01182 79.93501]
0
[77.108246 80.15604 ]
1
[80.0087  79.98667]
0
[77.09795 80.20355]
1
[80.00633  80.051506]
1
[79.69099

KeyboardInterrupt: 

In [12]:
env.close()

In [ ]:
# random action
for i in tqdm(range(50)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    for j in range(1000):
        env.render()
        observation,reward,done,info = env.step(env.action_space.sample())
        print(reward,done)
        if done:
            break